In [4]:
from dev_spark_session import DevSparkSession 
from get_stock_data import GetStockData
from scd_type2_handler import SCDType2Handler

stockdata = GetStockData() 
spark = DevSparkSession().spark


In [5]:
from pyspark.sql.functions import col, explode # sha2,concat_ws,trim, lit

In [6]:
json_data  = stockdata.get_historical_stock_data()


data saved for symbol QQQ and date range 2021-01-01 to 2021-12-31.
data saved for symbol VOO and date range 2021-01-01 to 2021-12-31.


In [7]:

# df = spark.createDataFrame(json_data) 

# df = df.withColumn("symbol", col("meta").symbol).filter( col("status") == "ok" ).drop(df.status)

# df.show(truncate = False)




In [8]:

# df.write \
#     .format("parquet") \
#     .mode("overwrite") \
#     .partitionBy( "symbol" ) \
#     .save("/Users/PC/Desktop/VS Code Repositories/azure-stock-market/Azure storage/Bronze")


### Bronze -> Silver

In [9]:

# from pyspark.sql.functions import current_timestamp, sha2, concat_ws
# from pyspark.sql.functions import max, col,cast,unhex, hex


# Paths for Bronze and Silver data
bronze_path = "/Users/PC/Desktop/VS Code Repositories/azure-stock-market/Azure storage/Bronze"  # Bronze data stored in Parquet or Delta
silver_path = "/Users/PC/Desktop/VS Code Repositories/azure-stock-market/Azure storage/Silver/delta-table"  # Target location for Silver Delta table

stage_path = "/Users/PC/Desktop/VS Code Repositories/azure-stock-market/Azure storage/Silver/stage-table"  # Target location for Silver Delta table



### Tranformt Bronze -> Silver

df_bronze = spark.read.load(bronze_path).drop("symbol")
df_bronze = df_bronze.withColumn( "values", explode("values") )

df_bronze = df_bronze.select(
    col("meta").getItem("symbol").alias("Symbol"),
    col("meta").getItem("exchange").alias("ExchangeName"),
    col("meta").getItem("currency").alias("Currency"),
    col("meta").getItem("type").alias("Type"),
    col("meta").getItem("exchange_timezone").alias("ExchangeTimeZone"),
    col("values").getItem("volume").alias("Volume"),
    col("values").getItem("high").alias("High"),
    col("values").getItem("low").alias("Low"),
    col("values").getItem("close").alias("Close"),
    col("values").getItem("open").alias("Open"),
    col("values").getItem("datetime").alias("Date"))

df_bronze = df_bronze.dropDuplicates()



df_bronze.show()


+------+------------+--------+----+----------------+--------+----------+---------+----------+----------+----------+
|Symbol|ExchangeName|Currency|Type|ExchangeTimeZone|  Volume|      High|      Low|     Close|      Open|      Date|
+------+------------+--------+----+----------------+--------+----------+---------+----------+----------+----------+
|   QQQ|      NASDAQ|     USD| ETF|America/New_York|48899500| 389.53000|380.20001| 389.20999|384.049988|2021-12-21|
|   QQQ|      NASDAQ|     USD| ETF|America/New_York|52545800| 398.28000|393.35999|398.010010| 396.62000|2021-12-10|
|   QQQ|      NASDAQ|     USD| ETF|America/New_York|36664900| 337.32001|332.92001| 337.10999| 333.67999|2021-04-09|
|   QQQ|      NASDAQ|     USD| ETF|America/New_York|33187200| 342.14999|339.17001| 340.14999| 341.92999|2021-04-27|
|   QQQ|      NASDAQ|     USD| ETF|America/New_York|39204000| 399.82001|396.22000| 399.60999| 398.17001|2021-12-08|
|   QQQ|      NASDAQ|     USD| ETF|America/New_York|59722700| 346.38000|

In [ ]:
parameters = {
        "businessColumns" : "Symbol,ExchangeName,Currency,Date",
        "typeIColumns" : "Symbol"
        }

scd2Handler =  SCDType2Handler(parameters)

scd2Handler.refresh_timestamp()

scd2Handler.businessColumnsList


# df_bronze_audit = df_bronze.transform(scd2Handler.add_audit_columns)
# df_bronze_audit.show()


1

In [ ]:
spark.sql( """
          
          describe history delta.`/Users/PC/Desktop/VS Code Repositories/azure-stock-market/Azure storage/Silver/delta-table`
          
          """).show(truncate=False)


In [ ]:
df_silver = spark.read.format("delta").load(silver_path)

df_silver.createOrReplaceTempView("silver") 

spark.sql("select __hashkey, * from silver order by __hashkey , __ActivationDateTime desc ").show(truncate=False)

In [ ]:
df = spark.read.format("delta").load("/Users/PC/Desktop/VS Code Repositories/azure-stock-market/Azure storage/Silver/delta-table")
df.schema

In [ ]:

# from pyspark.sql.types import StructType, StructField, DateType, StringType, DecimalType, IntegerType


# schema = StructType([StructField('Symbol', StringType(), True), 
#             StructField('ExchangeName', StringType(), True), 
#             StructField('Currency', StringType(), True), 
#             StructField('Type', StringType(), True), 
#             StructField('ExchangeTimeZone', StringType(), True), 
#             StructField('Volume', StringType(), True), 
#             StructField('High', StringType(), True), 
#             StructField('Low', StringType(), True), 
#             StructField('Close', StringType(), True), 
#             StructField('Open', StringType(), True), 
#             StructField('Date', StringType(), True)])

df_silver = spark.createDataFrame( df_bronze , schema = schema )

df_silver.show()

In [ ]:
df = spark.read.load("/Users/PC/Desktop/VS Code Repositories/azure-stock-market/Azure storage/Bronze"); 

df_dim = df.select(
    col("meta").getItem("symbol").alias("Symbol"),
    col("meta").getItem("exchange").alias("ExchangeName"),
    col("meta").getItem("currency").alias("Currency"),
    col("meta").getItem("type").alias("Type"),
    col("meta").getItem("exchange_timezone").alias("ExchangeTimezone")
    ) 


# df_dim = df_dim.withColumn("__HashKey", sha2( col("Symbol"), 256)) 
# df_dim = df_dim.withColumn("__HashKey", sha2( concat_ws("|",  *businessColumnsList), 256)) 

 
# profile data
df_dim.limit(1).show(truncate = False) 


TypeIColumnsList

 

In [ ]:

df_flat_exploded = df_flat2.select(
                        col("symbol"),
                        explode( "values").alias("record"))
                        
                        # .withColumn( "volume" , col("record").getItem("volume") ) \
                        # .withColumn( "high" , col("record").getItem("high") ) \
                        # .withColumn( "low" , col("record").getItem("low") ) \
                        # .withColumn( "close" , col("record").getItem("close") ) \
                        # .withColumn( "open" , col("record").getItem("open") ) \
                        # .withColumn( "date" , col("record").getItem("datetime") ) \
                        #         .drop("record")


df_flat_exploded =   df_flat_exploded.select(
        "symbol",
        col("record").getItem("volume").alias("volume"),
        col("record").getItem("high").alias("high"),
        col("record").getItem("low").alias("low"),
        col("record").getItem("close").alias("close"),
        col("record").getItem("open").alias("open"),
        col("record").getItem("date").alias("date")
        
  )   
df_flat_exploded = df_flat_exploded.filter(  col("symbol")  == "VOO" ).distinct().count()     


print(df_flat_exploded)
# df_flat_exploded.show(truncate = False )